In [12]:
import h5py
import time
import numpy as np 
import pandas as pd
import scipy.misc
from scipy import ndimage
import matplotlib.pyplot as plt
from PIL import Image

In [2]:
def get_2D_data_from_h5_filtered_np_v2(h5_path, part_name, slice_name, show_info = False):
    # opening h5 and getting the data
    start_time = time.time()

    with h5py.File(h5_path, 'r') as h5:
        # check whether slice exists -> if not: empty array returned
        if slice_name in h5[part_name]:
            Y_Axis = np.array(h5[part_name][slice_name]['Y-Axis'][:]).astype(int)
            Area = np.array(h5[part_name][slice_name]['Area'][:]).astype(int)
            Intensity = np.array(h5[part_name][slice_name]['Intensity'][:]).astype(int)
            X_Axis = np.array(h5[part_name][slice_name]['X-Axis'][:]).astype(int)

            X_Axis_size = X_Axis.size
            Y_Axis_size = Y_Axis.size
            Area_size = Area.size
            Intensity_size = Intensity.size

            # if dimensions aren't equal the following code block is entered
            if not X_Axis_size == Y_Axis_size == Area_size == Intensity_size:

                # determine the lowest value among the different sizes
                size_arr = np.array([X_Axis_size, Y_Axis_size, Area_size, Intensity_size])
                min_size = size_arr.min()

                if X_Axis_size != min_size:
                    diff_size_x = X_Axis_size - min_size  # calculating the difference between the actual value and the minimum and substracting it from the array
                    X_Axis_new = np.delete(X_Axis, -diff_size_x)
                    X_Axis = X_Axis_new
                    X_Axis_size = X_Axis.size

                if Y_Axis_size != min_size:
                    diff_size_y = Y_Axis_size - min_size
                    Y_Axis_new = np.delete(Y_Axis, -diff_size_y)
                    Y_Axis = Y_Axis_new
                    Y_Axis_size = Y_Axis.size

                if Area_size != min_size:
                    diff_size_area = Area_size - min_size
                    Area_new = np.delete(Area, -diff_size_area)
                    Area = Area_new
                    Area_size = Area.size

                if Intensity_size != min_size:
                    diff_size_intensity = Intensity_size - min_size
                    Intensity_new = np.delete(Intensity, -diff_size_intensity)
                    Intensity = Intensity_new
                    Intensity_size = Intensity.size

                # by reducing all the dimensions to the minimum equal dimensions are guaranteed
                # there is a risk of deleting more than just one datapoint without noticing -> maybe add an alert after more than 5(?) while iterations
            #

            if show_info:
                print(str(X_Axis_size) + ' total data points found')

            combos = np.stack((X_Axis, Y_Axis, Area, Intensity), axis=-1)

            # filtering out the outlier data points
            median_x = np.median(combos[:, 0])
            median_y = np.median(combos[:, 1])
            std_x = int(combos[:, 0].std())
            std_y = int(combos[:, 1].std())
            low_limit_x = median_x - 2 * std_x
            low_limit_y = median_y - 2 * std_y
            high_limit_x = median_x + 2 * std_x
            high_limit_y = median_y + 2 * std_y

            combos = np.delete(combos, np.where(combos[:, 0] < low_limit_x), axis=0)
            combos = np.delete(combos, np.where(combos[:, 0] > high_limit_x), axis=0)
            combos = np.delete(combos, np.where(combos[:, 1] < low_limit_y), axis=0)
            combos = np.delete(combos, np.where(combos[:, 1] > high_limit_y), axis=0)

            # filtering out the data points where area and intensity are =0
            area_zeros = np.where(combos[:, 2] == 0)
            intensity_zeros = np.where(combos[:, 3] == 0)
            zero_area_intensity_indices = np.intersect1d(area_zeros,
                                                         intensity_zeros)  # array of indices where area AND intensity are = 0

            # deleting all the datapoints where area AND intensity are = 0
            combos_wo_only_zeros = np.delete(combos, zero_area_intensity_indices, axis=0)
            if show_info:
                print(str(combos_wo_only_zeros.shape[0]) + ' data points where area != 0 AND intensity != 0')

            combos_wo_only_zeros_unique, unique_indices = np.unique(combos_wo_only_zeros[:, [0, 1]], axis=0,
                                                                    return_index=True)
            combos_unique = combos_wo_only_zeros[unique_indices]

            if show_info:
                print(str(combos_unique.shape[0]) + ' unique data points where area != 0 AND intensity != 0')

            Index_range = np.arange(combos_wo_only_zeros.shape[0])
            indices_of_interest = np.setdiff1d(Index_range,
                                               unique_indices)  # all the indices belonging to non unique x,y-combinations

            combo_processed_array = np.empty([0, 4], dtype=int)
            start_time = time.time()
            combos_wo_only_zeros_copy = np.copy(combos_wo_only_zeros)
            index_counter = 0
            shape_counter = 0
            indices_list = []

            if show_info:
                print("vor iterieren %s seconds ---" % (time.time() - start_time))

            for index in indices_of_interest:
                xy_combo = combos_wo_only_zeros[:, [0, 1]][index]
                if \
                np.where((combo_processed_array[:, 0] == xy_combo[0]) * (combo_processed_array[:, 1] == xy_combo[1]))[
                    0].size == 0:
                    index_counter += 1
                    xy_combo = combos_wo_only_zeros[:, [0, 1]][index]
                    indices_relevant = \
                    np.where((combos_wo_only_zeros[:, 0] == xy_combo[0]) * (combos_wo_only_zeros[:, 1] == xy_combo[1]))[
                        0]
                    max_area_of_combo = np.amax(combos_wo_only_zeros[:, 2][indices_relevant])
                    max_intensity_of_combo = np.amax(combos_wo_only_zeros[:, 3][indices_relevant])

                    max_combos = np.stack((xy_combo[0], xy_combo[1], max_area_of_combo, max_intensity_of_combo),
                                          axis=-1)

                    combos_wo_only_zeros_copy = np.vstack((combos_wo_only_zeros_copy, max_combos))
                    shape_counter += indices_relevant.shape[0]
                    indices_list.append(list(indices_relevant))

                    combo_processed_array = np.vstack((combo_processed_array, max_combos))

            indices_relevant = np.hstack(indices_list)
            combos_wo_only_zeros_copy = np.delete(combos_wo_only_zeros_copy, indices_relevant, axis=0)
        else:
            combos_wo_only_zeros_copy = np.empty([0, 4], dtype=int)
            print('{} is not existing -> empty array created'.format(slice_name))

        if show_info:
            print("array creation took %s seconds ---" % (time.time() - start_time))
        return combos_wo_only_zeros_copy


def get_min_max_values (h5_path, part_name,  max_slice_num):
    df = pd.DataFrame(columns=['Slice_num', 'minX', 'maxX', 'minY', 'maxY', 'diameterX', 'diameterY'])

    for num_slice in range(max_slice_num):
        slice_name = 'Slice' + str("{:05d}".format(num_slice + 1))

        array = get_2D_data_from_h5_filtered_np_v2(h5_path, part_name, slice_name)
        minX = array[:, 0].min()
        maxX = array[:, 0].max()
        minY = array[:, 1].min()
        maxY = array[:, 1].max()
        diameterX = maxX - minX
        diameterY = maxY - minY

        df = df.append({'Slice_num': "{:05d}".format(num_slice + 1), 'minX': minX, 'maxX': maxX, 'minY': minY, 'maxY': maxY,
                        'diameterX': diameterX, 'diameterY': diameterY}, ignore_index=True)

    return df['minX'].min(), df['minY'].min(), df['maxX'].max(), df['maxY'].max()


def dock_array_to_zero(array, minX, minY):
    if minX >= 0 and minY >= 0:
        array[:, 0] = array[:, 0] - minX
        array[:, 1] = array[:, 1] - minY
    elif minX < 0 and minY < 0:
        array[:, 0] = array[:, 0] + abs(minX)
        array[:, 1] = array[:, 1] + abs(minY)
    elif minX >= 0 and minY < 0:
        array[:, 0] = array[:, 0] - minX
        array[:, 1] = array[:, 1] + abs(minY)
    elif minX < 0 and minY >= 0:
        array[:, 0] = array[:, 0] + abs(minX)
        array[:, 1] = array[:, 1] - minY
    return array


In [3]:
h5_path = '/home/jan/Documents/Trainingsdaten/ZP_4/ZP_4_full_part.h5'
part_name = 'ZP4_combined'
slice_name = "Slice00001"

In [4]:
array_not_docked = get_2D_data_from_h5_filtered_np_v2(h5_path, part_name, slice_name, show_info = False)

In [5]:
minX, minY, maxX, maxY = get_min_max_values (h5_path, part_name,  1593)

In [6]:
array_docked = dock_array_to_zero(array_not_docked, minX, minY)

In [7]:
array_docked

array([[ 405,  148,    0,  373],
       [ 397,  156,    0,  577],
       [ 392,  161,    0,  705],
       ...,
       [1669, 1233, 1021, 1520],
       [1317, 1625,  739,  927],
       [1482, 1499,  866, 1130]])

In [8]:
df = pd.DataFrame({'x': array_docked[:, 0], 'y': array_docked[:, 1], 'area': array_docked[:, 2],'intensity': array_docked[:, 3]})

In [9]:
df

,x,y,area,intensity
0,405,148,0,373
1,397,156,0,577
2,392,161,0,705
3,385,168,0,745
4,380,173,315,773
...,...,...,...,...
9438,1714,1108,1212,1833
9439,1167,1695,1006,1107
9440,1669,1233,1021,1520
9441,1317,1625,739,927


In [10]:
i = 58

j = 20
print(i*j)
print((i+1)*j)

1160
1180


In [11]:
df[(df['x']>i*j) & (df['x']< (i+1)*j) & (df['y']> i*j) & (df['y']< (i+1)*j)]

,x,y,area,intensity
7496,1168,1176,1162,1709
7497,1173,1171,1162,1495


In [12]:
maxX

-15695

In [13]:
minX

-17441

In [14]:
maxX-minX

1746

In [15]:
1746/20

87.3

In [16]:
1746/0.252

6928.571428571428

In [20]:
from PIL import Image
import numpy as np

w, h = 224, 224
data = np.zeros((h, w), dtype=np.uint8)
data[0:100, 2:100] = 185# , 255, 255] # red patch in upper left
img = Image.fromarray(data,'L')#, 'RGB')
img.save('my.png')
img.show()

In [101]:
data

array([[  0,   0, 185, ...,   0,   0,   0],
       [  0,   0, 185, ...,   0,   0,   0],
       [  0,   0, 185, ...,   0,   0,   0],
       ...,
       [  0,   0,   0, ...,   0,   0,   0],
       [  0,   0,   0, ...,   0,   0,   0],
       [  0,   0,   0, ...,   0,   0,   0]], dtype=uint8)

In [19]:
df_Ausschnitt = df[(df['x']>pixels*counter) & (df['x']< pixels*(counter+1)) & (df['y']>pixels*counter) & (df['y']< (counter+1)*pixels)]

NameError: name 'pixels' is not defined

In [18]:
df_Ausschnitt['intensity'].max()

NameError: name 'df_Ausschnitt' is not defined

In [132]:
df_Ausschnitt['intensity']*255/2587

959     137.307692
960     140.067646
961     157.415926
1101     96.499807
1102     95.218400
           ...    
2604    136.814843
2605    175.257055
2606    151.797449
2607    191.028218
2608    151.107460
Name: intensity, Length: 200, dtype: float64

In [137]:
int_not_corrected = df_Ausschnitt['intensity'].values

In [143]:
int_corrected = (int_not_corrected*255/2587).astype(int)

In [144]:
int_corrected

array([137, 140, 157,  96,  95,  91,  95,  96,  96,  98,  98,  98, 142,
       142, 166, 154, 128, 124, 115, 126, 118, 105, 105, 106, 101, 100,
       108,  95, 126, 115, 104, 101,  95,  95,  96,  97,  90,  91,  92,
       100, 102, 105, 113,  99,  95,  92,  96,  86,  93, 113, 112, 116,
       103, 102, 139, 129, 170, 211, 205, 194, 247, 181, 189, 143, 133,
       122, 160, 129, 131, 115, 132, 151, 161, 139, 137, 137, 146,  91,
       104, 103,  99, 104, 100, 108, 114, 115, 118, 110, 105, 105, 116,
       105,  97,  94,  93, 103, 106, 139, 122, 119, 103,  98,  98,  95,
       100,  91,  89,  90,  92,  92, 111, 112, 115, 144, 226, 206, 153,
       171, 148, 173, 152, 132, 133, 126, 133, 138, 130, 144, 136, 162,
       174, 134, 123, 129, 149, 146, 139, 127, 200, 255, 188, 171,  94,
        97,  97,  94,  97,  92,  91,  92,  89,  90,  94,  85,  86,  90,
       109, 104, 101,  92,  95,  95,  93,  93, 103,  94, 125, 128, 130,
       135, 125, 186, 176, 150, 138, 142, 134, 154, 152, 127, 20

In [151]:
#Dataframe mit skalierten intensity Werten
df_Ausschnitt.loc[:,'intensity'] = int_corrected

In [165]:
df_Ausschnitt= df_Ausschnitt.drop(columns = [3])

In [166]:
df_Ausschnitt

,x,y,area,intensity
959,449,462,934,137
960,456,455,934,140
961,461,450,986,157
1101,501,450,859,96
1102,493,458,946,95
...,...,...,...,...
2604,641,668,979,136
2605,649,660,979,175
2606,654,655,980,151
2607,661,648,980,191


In [167]:
data_new = np.zeros((224, 224), dtype=np.uint8)

In [169]:
for index, row in df_Ausschnitt.iterrows():
    #print(row[0], 'is a', row[1])
    data_new[row[0]-pixels*counter,row[1]-pixels*counter] = row[3]

In [170]:
data_new

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8)

In [11]:
img = Image.fromarray(data_new,'L')#, 'RGB')
img.save('my.png')
img.show()

NameError: name 'Image' is not defined

In [62]:
pixels = 1600
counter = 0
w, h = pixels, pixels
area_int_switch = 3 #2 für area, 3 für intensity
print(pixels*counter)
print(pixels*(counter+1))

0
1600


In [78]:
#wiederholen für Schicht 2 und 3 für RGB-Bild
array_not_docked_1 = get_2D_data_from_h5_filtered_np_v2(h5_path, part_name, 'Slice00003', show_info = False)
array_not_docked_2 = get_2D_data_from_h5_filtered_np_v2(h5_path, part_name, 'Slice00004', show_info = False)
array_not_docked_3 = get_2D_data_from_h5_filtered_np_v2(h5_path, part_name, 'Slice00005', show_info = False)

array_docked_1 = dock_array_to_zero(array_not_docked_1, minX, minY)
array_docked_2 = dock_array_to_zero(array_not_docked_2, minX, minY)
array_docked_3 = dock_array_to_zero(array_not_docked_3, minX, minY)

df_1 = pd.DataFrame({'x': array_docked_1[:, 0], 'y': array_docked_1[:, 1], 'area': array_docked_1[:, 2],'intensity': array_docked_1[:, 3]})
df_2 = pd.DataFrame({'x': array_docked_2[:, 0], 'y': array_docked_2[:, 1], 'area': array_docked_2[:, 2],'intensity': array_docked_2[:, 3]})
df_3 = pd.DataFrame({'x': array_docked_3[:, 0], 'y': array_docked_3[:, 1], 'area': array_docked_3[:, 2],'intensity': array_docked_3[:, 3]})

df_Ausschnitt_1 = df_1[(df_1['x']>pixels*counter) & (df_1['x']< pixels*(counter+1)) & (df_1['y']>pixels*counter) & (df_1['y']< (counter+1)*pixels)]
df_Ausschnitt_2 = df_2[(df_2['x']>pixels*counter) & (df_2['x']< pixels*(counter+1)) & (df_2['y']>pixels*counter) & (df_2['y']< (counter+1)*pixels)]
df_Ausschnitt_3 = df_3[(df_3['x']>pixels*counter) & (df_3['x']< pixels*(counter+1)) & (df_3['y']>pixels*counter) & (df_3['y']< (counter+1)*pixels)]

int_not_corrected_1 = df_Ausschnitt_1['intensity'].values
int_not_corrected_2 = df_Ausschnitt_2['intensity'].values
int_not_corrected_3 = df_Ausschnitt_3['intensity'].values

area_not_corrected_1 = df_Ausschnitt_1['area'].values
area_not_corrected_2 = df_Ausschnitt_2['area'].values
area_not_corrected_3 = df_Ausschnitt_3['area'].values

int_corrected_1 = (int_not_corrected_1*255/df_Ausschnitt_1['intensity'].max()).astype(int)
int_corrected_2 = (int_not_corrected_2*255/df_Ausschnitt_2['intensity'].max()).astype(int)
int_corrected_3 = (int_not_corrected_3*255/df_Ausschnitt_3['intensity'].max()).astype(int)

area_corrected_1 = (area_not_corrected_1*255/df_Ausschnitt_1['area'].max()).astype(int)
area_corrected_2 = (area_not_corrected_2*255/df_Ausschnitt_2['area'].max()).astype(int)
area_corrected_3 = (area_not_corrected_3*255/df_Ausschnitt_3['area'].max()).astype(int)

df_Ausschnitt_1.loc[:,'intensity'] = int_corrected_1
df_Ausschnitt_2.loc[:,'intensity'] = int_corrected_2
df_Ausschnitt_3.loc[:,'intensity'] = int_corrected_3

In [79]:
data_new_1 = np.zeros((pixels, pixels), dtype=np.uint8)
data_new_2 = np.zeros((pixels, pixels), dtype=np.uint8)
data_new_3 = np.zeros((pixels, pixels), dtype=np.uint8)

for index, row in df_Ausschnitt_1.iterrows():
    #print(row[0], 'is a', row[1])
    data_new_1[row[0]-pixels*counter,row[1]-pixels*counter] = row[3]

for index, row in df_Ausschnitt_2.iterrows():
    #print(row[0], 'is a', row[1])
    data_new_2[row[0]-pixels*counter,row[1]-pixels*counter] = row[3]
    
for index, row in df_Ausschnitt_3.iterrows():
    #print(row[0], 'is a', row[1])
    data_new_3[row[0]-pixels*counter,row[1]-pixels*counter] = row[3]

In [47]:
img = Image.fromarray(data_new_1,'L')#, 'RGB')
img.save('my.png')
img.show()

In [25]:
blurred_data_new_1 = ndimage.gaussian_filter(data_new_1, sigma = 2)



img = Image.fromarray(blurred_data_new_1,'L')
img.save('my.png')
img.show()




In [85]:
new = ndimage.grey_dilation(data_new_1, size=(10,10))#, structure=np.ones((10, 10)))
img = Image.fromarray(new,'L')
img.save('my.png')
img.show()



In [51]:
np.random.seed(2)
im = np.zeros((64, 64))
x, y = (63*np.random.random((2, 8))).astype(np.int)
im[x, y] = np.arange(8)

bigger_points = ndimage.grey_dilation(im, size=(5, 5))#, structure=np.ones((5, 5)))

square = np.zeros((16, 16))
square[4:-4, 4:-4] = 1
dist = ndimage.distance_transform_bf(square)
dilate_dist = ndimage.grey_dilation(dist, size=(3, 3), \
        structure=np.ones((3, 3)))

In [50]:
dist = ndimage.distance_transform_bf(data_new_1)
dilate_dist = ndimage.grey_dilation(dist, size=(3, 3), \
        structure=np.ones((3, 3)))
img = Image.fromarray(dist,'L')

img.save('my.png')
img.show()

In [377]:
img = Image.fromarray(data_new_2,'L')#, 'RGB')
img.save('my.png')
img.show()

In [378]:
img = Image.fromarray(data_new_3,'L')#, 'RGB')
img.save('my.png')
img.show()

In [80]:
data_new_1 = ndimage.grey_dilation(data_new_1, size=(10,10))
data_new_2 = ndimage.grey_dilation(data_new_2, size=(10,10))
data_new_3 = ndimage.grey_dilation(data_new_3, size=(10,10))

In [81]:
data = np.zeros((h, w, 3), dtype=np.uint8)

In [82]:
for i in range(pixels):
    for j in range(pixels):
        data[i][j][0] = data_new_1[i][j]
        data[i][j][1] = data_new_2[i][j]
        data[i][j][2] = data_new_3[i][j]

In [74]:
data

array([[[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       ...,

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]]], dtype=uint8)

In [83]:
img = Image.fromarray(data, 'RGB')
img.save('my.png')
img.show()

In [29]:
df_1

,x,y,area,intensity
0,1300,413,0,426
1,1305,418,0,631
2,1312,425,84,760
3,1317,430,168,833
4,1325,438,168,890
...,...,...,...,...
9797,281,1087,1323,1444
9798,623,1450,735,1313
9799,315,1161,795,1120
9800,541,1407,671,802


In [30]:
array_docked_1

array([[1300,  413,    0,  426],
       [1305,  418,    0,  631],
       [1312,  425,   84,  760],
       ...,
       [ 315, 1161,  795, 1120],
       [ 541, 1407,  671,  802],
       [ 387, 1273,  508, 1048]])

In [39]:
#trying to save array_docked_1 to disk
np.save('/home/jan/Documents/Trainingsdaten/Saved_NP_arrays/array_docked_1',data)

In [40]:
array_loaded = np.load('/home/jan/Documents/Trainingsdaten/Saved_NP_arrays/array_docked_1.npy')

In [41]:
array_loaded

array([[[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       ...,

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]]], dtype=uint8)

In [42]:
array_loaded.shape

(224, 224, 3)